<a href="https://colab.research.google.com/github/nhs-research/parasitic-nematode-EfficientNetV2B0/blob/main/Parasitic_EfficientNetV2B0_MultiAugment_Keras3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
!pip install keras-cv

In [ ]:
from google.colab import drive

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn

import sklearn
import os

import sklearn.metrics

import keras
import keras.applications as applications
import keras.layers as layers
import keras.losses as losses
import keras.optimizers as optimizers
import keras.callbacks as callbacks
import keras.models as models
import keras.backend as K

import keras_cv

import scipy.ndimage.filters as filters
import scipy.ndimage.morphology as morphology
import math

import warnings

import zipfile

import os

## Colab .env

In [ ]:
from google.colab import userdata

ZIP_PATH = userdata.get('ZIP_PATH')
DATASET_NAME = userdata.get('DATASET_NAME')
CHECKPOINT_PATH = userdata.get('CHECKPOINT_PATH')

print(ZIP_PATH)
print(DATASET_NAME)
print(CHECKPOINT_PATH)

/gdrive/MyDrive/gilbert/NemaApp_Retrain/Parasitic/Microscopic Image Dataset of Plant-Parasitic Nematodes.zip
Microscopic Image Dataset of Plant-Parasitic Nematodes


## Create model output folder

In [ ]:
os.makedirs('/content/models')

## Helper func

In [ ]:
def calculate_metrics(model, test, ds, num_classes):
  # get the labels
  predictions = np.array([])
  labels =  np.array([])

  for x, y in test:
    predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis=-1)])
    labels = np.concatenate([labels, y.numpy()])

  # [Metric 1] confusion matrix
  conf_matrix = tf.math.confusion_matrix(labels, predictions).numpy()
  conf_matrix_df = pd.DataFrame(conf_matrix,
                     index = ds.class_names,
                     columns = ds.class_names)
  plt.figure(figsize=(14,14))
  ax = sn.heatmap(conf_matrix_df, cmap='Blues', annot=True, fmt='d')
  ax.set_xlabel('\nPredicted Classes')
  ax.set_ylabel('Actual Classes');
  plt.show()

  # [Metric 2] Overall test accuracy
  overall_accuracy = np.diagonal(conf_matrix_df).sum()/(conf_matrix_df.sum().sum())*100
  print('Test accuracy: {:.4f}%'.format(overall_accuracy))

  # [Metric 3] Mean class accuracy
  pred_classes = np.diagonal(conf_matrix)
  actual_classes = conf_matrix.sum(axis=1) # sum by rows
  datasize = num_classes
  sumAccuracy = 0
  divisor = 0
  for i in range(datasize):
    if actual_classes[i] != 0:
      acc = pred_classes[i]/actual_classes[i]
      sumAccuracy += acc
      divisor = divisor+1

  meanClassAccuracy = 0
  if divisor > 0:
    meanClassAccuracy = sumAccuracy/divisor
  print('Mean class accuracy: {:.4f}'.format(meanClassAccuracy))

  # [Metric 4] F1 score
  f1score = sklearn.metrics.f1_score(labels, predictions, average="weighted")
  print('F1 Score: {:.4f}'.format(f1score))

  # [Metric 5] Weighted average Precision and Recall
  precisionScore = sklearn.metrics.precision_score(labels, predictions, average="weighted")
  recallScore = sklearn.metrics.recall_score(labels, predictions, average="weighted")
  print('Avg Precision Score: {:.4f}'.format(precisionScore))
  print('Avg Recall Score: {:.4f}'.format(recallScore))


In [ ]:
def plot_history(hist, dataset, loss, augment):
  plt.plot(hist.history['accuracy'])
  plt.plot(hist.history['val_accuracy'])
  ttPerformance = "[PERFORMANCE] " + dataset + " data | " + loss + " | " + augment
  plt.title(ttPerformance)
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(hist.history['loss'])
  plt.plot(hist.history['val_loss'])
  ttLoss = "[LOSS] " + dataset + " data | " + loss + " | " + augment
  plt.title(ttLoss)
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

# Model EfficientNetV2B0

## Data prep

In [ ]:
drive.mount('/gdrive/')

Mounted at /gdrive/


In [ ]:
zip_ref = zipfile.ZipFile(ZIP_PATH, 'r')
zip_ref.extractall('/content/Dataset/')

In [ ]:
local_dataset_path = os.path.join('/content/Dataset/', DATASET_NAME)
print(local_dataset_path)

/content/Dataset/Microscopic Image Dataset of Plant-Parasitic Nematodes


In [ ]:
img_w = 224
img_h = 224

mix_classes = 26
local_classes = 11

def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

# crop mix
# mix_ds = tf.keras.utils.image_dataset_from_directory(
#     cropped_dataset_path,
#     image_size=(img_h, img_w),
#     batch_size=None, shuffle=False)

# mix_size = tf.data.experimental.cardinality(mix_ds).numpy()
# train_mix_ds, val_mix_ds, ttest_mix_ds = get_dataset_partitions_tf(mix_ds, mix_size, train_split=0.8, val_split=0.1, test_split=0.1)

# crop local
local_ds = tf.keras.utils.image_dataset_from_directory(
    local_dataset_path,
    image_size=(img_h, img_w),
    batch_size=None, shuffle=False)

local_size = tf.data.experimental.cardinality(local_ds).numpy()
train_local_ds, val_local_ds, ttest_local_ds = get_dataset_partitions_tf(local_ds, local_size, train_split=0.8, val_split=0.1, test_split=0.1)


Found 1016 files belonging to 11 classes.


In [ ]:
# preprocess
# input between -1 and 1
# shape [224, 224, 3]
# batch size 32
ep = 60
ep_local = 100
bs = 32

AUTOTUNE = tf.data.AUTOTUNE
rng = tf.random.Generator.from_seed(123, alg='philox')

def preprocess_efficientnet(image, label):
  image = tf.image.grayscale_to_rgb(tf.image.rgb_to_grayscale(image))
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = (image - 127.5) / 127.5  # value to -1:1
  return image, label

def random_flip(image, label):
  seed = rng.make_seeds(2)[0]
  image = tf.image.stateless_random_flip_left_right(image, seed=seed)
  image = tf.image.stateless_random_flip_up_down(image, seed=seed)
  return image, label

def random_brightness(image, label):
  seed = rng.make_seeds(2)[0]
  image = tf.image.stateless_random_brightness(image, 0.3, seed=seed)
  image = tf.clip_by_value(image, -1, 1) # clip to range -1:1
  return image, label

def random_gaussian_blur(image, label):
  chance = rng.uniform(shape=[], minval=0, maxval=1)
  if chance > 0.5: # 50% chance
    # image = tfa.image.gaussian_filter2d(image, filter_shape=(3,3))
    blur_layer = keras_cv.layers.RandomGaussianBlur(kernel_size=3, factor=1)
    image = blur_layer(image)
    image = tf.clip_by_value(image, -1, 1) # clip to range -1:1
  return image, label

def random_gaussian_noise(image, label):
  chance = rng.uniform(shape=[], minval=0, maxval=1)
  if chance > 0.5: # 50% chance
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.15, dtype=tf.float32)
    image = tf.add(image, noise)
    image = tf.clip_by_value(image, -1, 1) # clip to range -1:1
    image = tf.image.grayscale_to_rgb(tf.image.rgb_to_grayscale(image))
  return image, label

def random_contrast(image, label):
  seed = rng.make_seeds(2)[0]
  image = tf.image.stateless_random_contrast(image, 0.3, 3, seed=seed)
  image = tf.clip_by_value(image, -1, 1) # clip to range -1:1
  return image, label

# ------ MIX ------
# # planned combination of
# # [v] [brightness / blur / noise] = 3 dataset

# train_mix_brightness_blur = train_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
#         random_brightness, num_parallel_calls=AUTOTUNE).map(
#             random_gaussian_blur, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# train_mix_brightness_noise = train_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
#         random_brightness, num_parallel_calls=AUTOTUNE).map(
#             random_gaussian_noise, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# train_mix_blur_noise = train_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
#         random_gaussian_blur, num_parallel_calls=AUTOTUNE).map(
#             random_gaussian_noise, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# # [v] [brightness / blur / noise / contrast] = 6 dataset

# train_mix_brightness_contrast = train_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
#         random_brightness, num_parallel_calls=AUTOTUNE).map(
#             random_contrast, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# train_mix_blur_contrast = train_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
#         random_gaussian_blur, num_parallel_calls=AUTOTUNE).map(
#             random_contrast, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# train_mix_noise_contrast = train_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
#         random_gaussian_noise, num_parallel_calls=AUTOTUNE).map(
#             random_contrast, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# ### VALIDATION AND TEST
# val_mix_normal = val_mix_ds.map(
#     preprocess_efficientnet, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# test_mix_ds = ttest_mix_ds.map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE).batch(bs)

# ========================================[local]==================================================

# planned combination of
# [v] [brightness / blur / noise] = 3 dataset

train_local_brightness_blur = train_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
        random_brightness, num_parallel_calls=AUTOTUNE).map(
            random_gaussian_blur, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

train_local_brightness_noise = train_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
        random_brightness, num_parallel_calls=AUTOTUNE).map(
            random_gaussian_noise, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

train_local_blur_noise = train_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
        random_gaussian_blur, num_parallel_calls=AUTOTUNE).map(
            random_gaussian_noise, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

# [v] [brightness / blur / noise / contrast] = 6 dataset

train_local_brightness_contrast = train_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
        random_brightness, num_parallel_calls=AUTOTUNE).map(
            random_contrast, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

train_local_blur_contrast = train_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
        random_gaussian_blur, num_parallel_calls=AUTOTUNE).map(
            random_contrast, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

train_local_noise_contrast = train_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).map(
        random_gaussian_noise, num_parallel_calls=AUTOTUNE).map(
            random_contrast, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

### VALIDATION AND TEST
val_local_normal = val_local_ds.map(
    preprocess_efficientnet, num_parallel_calls=AUTOTUNE).prefetch(buffer_size=AUTOTUNE).batch(bs).cache().shuffle(1000)

test_local_ds = ttest_local_ds.map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE).batch(bs)



## TRAINING

### adam LOSS

#### Brightness + Blur

In [ ]:
dir_brightness_blur = os.path.join(CHECKPOINT_PATH, "adam/brightness_blur/cp_epoch_{epoch:02d}.weights.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_blur,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_adam_brightness_blur = tf.keras.Model(input, output)

# compile loss func
local_adam_brightness_blur.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
local_adam_brightness_blur_hist = local_adam_brightness_blur.fit(
  train_local_brightness_blur,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping],
  verbose=1
)

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 463s 12s/step - accuracy: 0.2040 - loss: 4.6091 - val_accuracy: 0.5743 - val_loss: 1.4207
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 320s 13s/step - accuracy: 0.7326 - loss: 0.8712 - val_accuracy: 0.7327 - val_loss: 0.9267
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 309s 12s/step - accuracy: 0.9138 - loss: 0.2975 - val_accuracy: 0.7822 - val_loss: 0.6176
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 209s 8s/step - accuracy: 0.9698 - loss: 0.1040 - val_accuracy: 0.8911 - val_loss: 0.3609
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 200s 7s/step - accuracy: 0.9948 - loss: 0.0382 - val_accuracy: 0.8614 - val_loss: 0.4250
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 198s 8s/step - accuracy: 0.9863 - loss: 0.0374 - val_accuracy: 0.9307 - val_loss: 0.3023
Epoch 7/100
 6/26 ━━━━━━━━━━━━━━━━━━━━ 2:31 8s/step - accuracy: 0.9942 - loss: 0.0113

In [ ]:
local_adam_brightness_blur.save_weights(os.path.join(CHECKPOINT_PATH, "adam/brightness_blur/best.weights.h5"))
plot_history(local_adam_brightness_blur_hist, "local", "adam", "brightness_blur")

In [ ]:
calculate_metrics(local_adam_brightness_blur, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | adam | brightness_blur]')

In [ ]:
local_adam_brightness_blur.save('/content/models/EffV2B0-adam-bright-blur.keras')

#### Brightness + Noise

In [ ]:
dir_brightness_noise = os.path.join(CHECKPOINT_PATH, "adam/brightness_noise/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_noise,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_adam_brightness_noise = tf.keras.Model(input, output)

# compile loss func
local_adam_brightness_noise.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_adam_brightness_noise_hist = local_adam_brightness_noise.fit(
  train_local_brightness_noise,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_adam_brightness_noise.save_weights(os.path.join(CHECKPOINT_PATH, "adam/brightness_noise/best.weights.h5"))

plot_history(local_adam_brightness_noise_hist, "local", "adam", "brightness_noise")

In [ ]:
calculate_metrics(local_adam_brightness_noise, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | adam | brightness_noise]')

In [ ]:
local_adam_brightness_noise.save('/content/models/EffV2B0-adam-bright-noise.keras')

#### Blur + Noise

In [ ]:
dir_blur_noise = os.path.join(CHECKPOINT_PATH, "adam/blur_noise/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_blur_noise,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_adam_blur_noise = tf.keras.Model(input, output)

# compile loss func
local_adam_blur_noise.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_adam_blur_noise_hist = local_adam_blur_noise.fit(
  train_local_blur_noise,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_adam_blur_noise.save_weights(os.path.join(CHECKPOINT_PATH, "adam/blur_noise/best.weights.h5"))

plot_history(local_adam_blur_noise_hist, "local", "adam", "blur_noise")

In [ ]:
calculate_metrics(local_adam_blur_noise, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | adam | blur_noise]')

In [ ]:
local_adam_blur_noise.save('/content/models/EffV2B0-adam-blur-noise.keras')

#### Brightness + Contrast

In [ ]:
dir_brightness_contrast = os.path.join(CHECKPOINT_PATH, "adam/brightness_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_adam_brightness_contrast = tf.keras.Model(input, output)

# compile loss func
local_adam_brightness_contrast.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_adam_brightness_contrast_hist = local_adam_brightness_contrast.fit(
  train_local_brightness_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_adam_brightness_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "adam/brightness_contrast/best.weights.h5"))

plot_history(local_adam_brightness_contrast_hist, "local", "adam", "brightness_contrast")

In [ ]:
calculate_metrics(local_adam_brightness_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | adam | brightness_contrast]')

In [ ]:
local_adam_brightness_contrast.save('/content/models/EffV2B0-adam-bright-contrast.keras')

#### Blur + Contrast

In [ ]:
dir_blur_contrast = os.path.join(CHECKPOINT_PATH, "adam/blur_contrast/cp_epoch_{epoch:02d}.weights.h5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_blur_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_adam_blur_contrast = tf.keras.Model(input, output)

# compile loss func
local_adam_blur_contrast.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_adam_blur_contrast_hist = local_adam_blur_contrast.fit(
  train_local_blur_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_adam_blur_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "adam/blur_contrast/best.weights.h5"))

plot_history(local_adam_blur_contrast_hist, "local", "adam", "blur_contrast")

In [ ]:
calculate_metrics(local_adam_blur_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | adam | blur_contrast]')

In [ ]:
local_adam_blur_contrast.save('/content/models/EffV2B0-adam-blur-contrast.keras')

#### Noise + Contrast

In [ ]:
dir_noise_contrast = os.path.join(CHECKPOINT_PATH, "adam/noise_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_noise_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_adam_noise_contrast = tf.keras.Model(input, output)

# compile loss func
local_adam_noise_contrast.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_adam_noise_contrast_hist = local_adam_noise_contrast.fit(
  train_local_noise_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_adam_noise_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "adam/noise_contrast/best.weights.h5"))

plot_history(local_adam_noise_contrast_hist, "local", "adam", "noise_contrast")

In [ ]:
calculate_metrics(local_adam_noise_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | adam | noise_contrast]')

In [ ]:
local_adam_noise_contrast.save('/content/models/EffV2B0-adam-noise-contrast.keras')

### sgd LOSS

#### Brightness + Blur

In [ ]:
dir_blur_contrast = os.path.join(CHECKPOINT_PATH, "sgd/brightness_blur/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_blur,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_sgd_brightness_blur = tf.keras.Model(input, output)

# compile loss func
local_sgd_brightness_blur.compile(
    optimizer='sgd',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_sgd_brightness_blur_hist = local_sgd_brightness_blur.fit(
  train_local_brightness_blur,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_sgd_brightness_blur.save_weights(os.path.join(CHECKPOINT_PATH, "sgd/brightness_blur/best.weights.h5"))

plot_history(local_sgd_brightness_blur_hist, "local", "sgd", "brightness_blur")

In [ ]:
calculate_metrics(local_sgd_brightness_blur, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | sgd | brightness_blur]')

In [ ]:
local_sgd_brightness_blur.save('/content/models/EffV2B0-sgd-bright-blur.keras')

#### Brightness + Noise

In [ ]:
dir_brightness_noise = os.path.join(CHECKPOINT_PATH, "sgd/brightness_noise/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_noise,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_sgd_brightness_noise = tf.keras.Model(input, output)

# compile loss func
local_sgd_brightness_noise.compile(
    optimizer='sgd',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_sgd_brightness_noise_hist = local_sgd_brightness_noise.fit(
  train_local_brightness_noise,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_sgd_brightness_noise.save_weights(os.path.join(CHECKPOINT_PATH, "sgd/brightness_noise/best.weights.h5"))

plot_history(local_sgd_brightness_noise_hist, "local", "sgd", "brightness_noise")

In [ ]:
calculate_metrics(local_sgd_brightness_noise, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | sgd | brightness_noise]')

In [ ]:
local_sgd_brightness_noise.save('/content/models/EffV2B0-sgd-bright-noise.keras')

#### Blur + Noise

In [ ]:
dir_blur_noise = os.path.join(CHECKPOINT_PATH, "sgd/blur_noise/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_blur_noise,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_sgd_blur_noise = tf.keras.Model(input, output)

# compile loss func
local_sgd_blur_noise.compile(
    optimizer='sgd',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_sgd_blur_noise_hist = local_sgd_blur_noise.fit(
  train_local_blur_noise,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_sgd_blur_noise.save_weights(os.path.join(CHECKPOINT_PATH, "sgd/blur_noise/best.weights.h5"))

plot_history(local_sgd_blur_noise_hist, "local", "sgd", "blur_noise")

In [ ]:
calculate_metrics(local_sgd_blur_noise, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | sgd | blur_noise]')

In [ ]:
local_sgd_blur_noise.save('/content/models/EffV2B0-sgd-blur-noise.keras')

#### Brightness + Contrast

In [ ]:
dir_brightness_contrast = os.path.join(CHECKPOINT_PATH, "sgd/brightness_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_sgd_brightness_contrast = tf.keras.Model(input, output)

# compile loss func
local_sgd_brightness_contrast.compile(
    optimizer='sgd',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_sgd_brightness_contrast_hist = local_sgd_brightness_contrast.fit(
  train_local_brightness_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_sgd_brightness_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "sgd/brightness_contrast/best.weights.h5"))

plot_history(local_sgd_brightness_contrast_hist, "local", "sgd", "brightness_contrast")

In [ ]:
calculate_metrics(local_sgd_brightness_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | sgd | brightness_contrast]')

In [ ]:
local_sgd_brightness_contrast.save('/content/models/EffV2B0-sgd-bright-contrast.keras')

#### Blur + Contrast

In [ ]:
dir_blur_contrast = os.path.join(CHECKPOINT_PATH, "sgd/blur_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_blur_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_sgd_blur_contrast = tf.keras.Model(input, output)

# compile loss func
local_sgd_blur_contrast.compile(
    optimizer='sgd',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_sgd_blur_contrast_hist = local_sgd_blur_contrast.fit(
  train_local_blur_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_sgd_blur_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "sgd/blur_contrast/best.weights.h5"))
plot_history(local_sgd_blur_contrast_hist, "local", "sgd", "blur_contrast")

In [ ]:
calculate_metrics(local_sgd_blur_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | sgd | blur_contrast]')

In [ ]:
local_sgd_blur_contrast.save('/content/models/EffV2B0-sgd-blur-contrast.keras')

#### Noise + Contrast

In [ ]:
dir_blur_contrast = os.path.join(CHECKPOINT_PATH, "sgd/noise_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_noise_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_sgd_noise_contrast = tf.keras.Model(input, output)

# compile loss func
local_sgd_noise_contrast.compile(
    optimizer='sgd',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_sgd_noise_contrast_hist = local_sgd_noise_contrast.fit(
  train_local_noise_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_sgd_noise_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "sgd/noise_contrast/best.weights.h5"))

plot_history(local_sgd_noise_contrast_hist, "local", "sgd", "noise_contrast")

In [ ]:
calculate_metrics(local_sgd_noise_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | sgd | noise_contrast]')

In [ ]:
local_sgd_noise_contrast.save('/content/models/EffV2B0-sgd-noise-contrast.keras')

### rmsprop LOSS

#### Brightness + Blur

In [ ]:
dir_brightness_blur = os.path.join(CHECKPOINT_PATH, "rmsprop/brightness_blur/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_blur,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_rmsprop_brightness_blur = tf.keras.Model(input, output)

# compile loss func
local_rmsprop_brightness_blur.compile(
    optimizer='rmsprop',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_rmsprop_brightness_blur_hist = local_rmsprop_brightness_blur.fit(
  train_local_brightness_blur,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_rmsprop_brightness_blur.save_weights(os.path.join(CHECKPOINT_PATH, "rmsprop/brightness_blur/best.weights.h5"))

plot_history(local_rmsprop_brightness_blur_hist, "local", "rmsprop", "brightness_blur")

In [ ]:
calculate_metrics(local_rmsprop_brightness_blur, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | rmsprop | brightness_blur]')

In [ ]:
local_rmsprop_brightness_blur.save('/content/models/EffV2B0-rms-bright-blur.keras')

#### Brightness + Noise

In [ ]:
dir_brightness_noise = os.path.join(CHECKPOINT_PATH, "rmsprop/brightness_noise/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_noise,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_rmsprop_brightness_noise = tf.keras.Model(input, output)

# compile loss func
local_rmsprop_brightness_noise.compile(
    optimizer='rmsprop',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_rmsprop_brightness_noise_hist = local_rmsprop_brightness_noise.fit(
  train_local_brightness_noise,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_rmsprop_brightness_noise.save_weights(os.path.join(CHECKPOINT_PATH, "rmsprop/brightness_noise/best.weights.h5"))

plot_history(local_rmsprop_brightness_noise_hist, "local", "rmsprop", "brightness_noise")

In [ ]:
calculate_metrics(local_rmsprop_brightness_noise, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | rmsprop | brightness_noise]')

In [ ]:
local_rmsprop_brightness_noise.save('/content/models/EffV2B0-rms-bright-noise.keras')

#### Blur + Noise

In [ ]:
dir_brightness_noise = os.path.join(CHECKPOINT_PATH, "rmsprop/blur_noise/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_blur_noise,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_rmsprop_blur_noise = tf.keras.Model(input, output)

# compile loss func
local_rmsprop_blur_noise.compile(
    optimizer='rmsprop',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_rmsprop_blur_noise_hist = local_rmsprop_blur_noise.fit(
  train_local_blur_noise,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_rmsprop_blur_noise.save_weights(os.path.join(CHECKPOINT_PATH, "rmsprop/blur_noise/best.weights.h5"))

plot_history(local_rmsprop_blur_noise_hist, "local", "rmsprop", "blur_noise")

In [ ]:
calculate_metrics(local_rmsprop_blur_noise, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | rmsprop | blur_noise]')

In [ ]:
local_rmsprop_blur_noise.save('/content/models/EffV2B0-rms-blur-noise.keras')

#### Brightness + Contrast

In [ ]:
dir_brightness_contrast = os.path.join(CHECKPOINT_PATH, "rmsprop/brightness_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_brightness_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_rmsprop_brightness_contrast = tf.keras.Model(input, output)

# compile loss func
local_rmsprop_brightness_contrast.compile(
    optimizer='rmsprop',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_rmsprop_brightness_contrast_hist = local_rmsprop_brightness_contrast.fit(
  train_local_brightness_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_rmsprop_brightness_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "rmsprop/brightness_contrast/best.weights.h5"))

plot_history(local_rmsprop_brightness_contrast_hist, "local", "rmsprop", "brightness_contrast")

In [ ]:
calculate_metrics(local_rmsprop_brightness_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | rmsprop | brightness_contrast]')

In [ ]:
local_rmsprop_brightness_contrast.save('/content/models/EffV2B0-rms-bright-contrast.keras')

#### Blur + Contrast

In [ ]:
dir_blur_contrast = os.path.join(CHECKPOINT_PATH, "rmsprop/blur_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_blur_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_rmsprop_blur_contrast = tf.keras.Model(input, output)

# compile loss func
local_rmsprop_blur_contrast.compile(
    optimizer='rmsprop',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_rmsprop_blur_contrast_hist = local_rmsprop_blur_contrast.fit(
  train_local_blur_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_rmsprop_blur_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "rmsprop/blur_contrast/best.weights.h5"))

plot_history(local_rmsprop_blur_contrast_hist, "local", "rmsprop", "blur_contrast")

In [ ]:
calculate_metrics(local_rmsprop_blur_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | rmsprop | blur_contrast]')

In [ ]:
local_rmsprop_blur_contrast.save('/content/models/EffV2B0-rms-blur-contrast.keras')

#### Noise + Contrast

In [ ]:
dir_noise_contrast = os.path.join(CHECKPOINT_PATH, "rmsprop/noise_contrast/cp_epoch_{epoch:02d}.weights.h5")

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=dir_noise_contrast,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=20*bs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=40, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# initiate model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling="max",include_preprocessing=False)
input = tf.keras.Input(shape=(224,224,3))
x = base_model(input)
output = tf.keras.layers.Dense(local_classes, activation="softmax")(x)
local_rmsprop_noise_contrast = tf.keras.Model(input, output)

# compile loss func
local_rmsprop_noise_contrast.compile(
    optimizer='rmsprop',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
local_rmsprop_noise_contrast_hist = local_rmsprop_noise_contrast.fit(
  train_local_noise_contrast,
  epochs = ep_local,
  validation_data= val_local_normal,
  callbacks=[checkpoint, early_stopping]
)

In [ ]:
local_rmsprop_noise_contrast.save_weights(os.path.join(CHECKPOINT_PATH, "rmsprop/noise_contrast/best.weights.h5"))

plot_history(local_rmsprop_noise_contrast_hist, "local", "rmsprop", "noise_contrast")

In [ ]:
calculate_metrics(local_rmsprop_noise_contrast, test_local_ds, local_ds, local_classes)
print('Metric Result - efficientnetM [local data | rmsprop | noise_contrast]')

In [ ]:
local_rmsprop_noise_contrast.save('/content/models/EffV2B0-rms-noise-contrast.keras')